In [ ]:
%load_ext autoreload
%autoreload 2

In [74]:
import sys, os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.metrics import roc_auc_score

from Pearson.pearson import Pearson

sys.path.append(os.getcwd())

In [75]:
import random

def sample_with_proportion(total, targets, proportion):
    total_sample = 10
    num_special = int(total_sample * proportion)
    special = np.random.choice(targets, num_special, replace=False)

    remaining = list(set(total) - set(targets))
    normal = np.random.choice(remaining, total_sample - num_special, replace=False)

    final = np.concatenate([special, normal])
    np.random.shuffle(final)
    return final

In [101]:
imp_dir = os.path.join(os.getcwd(), 'imputations')
for i in range(1, 4):
    gt_file = None
    if i == 1:
        gt_file = './SERGIO/data_sets/De-noised_100G_9T_300cPerT_4_DS1/gt_GRN.csv'
    elif i == 2:
        gt_file = './SERGIO/data_sets/De-noised_400G_9T_300cPerT_5_DS2/gt_GRN.csv'
    elif i == 3:
        gt_file = 'SERGIO/data_sets/De-noised_1200G_9T_300cPerT_6_DS3/gt_GRN.csv'
    
    # For now, just load clean data, save as csv, and load it back in
    load_dir = os.path.join(imp_dir, f'DS{i}')
    clean_npy = np.load(load_dir + '/DS6_clean.npy')
    df = pd.DataFrame(clean_npy)
    df.to_csv(load_dir + '/DS6_clean.csv', index=False)
    clean_df = pd.read_csv(os.path.join(load_dir, 'DS6_clean.csv'))
    
    # Read ground truth
    gt = pd.read_csv(gt_file, header=None)
    confirmed_genes = gt[0].unique()
    sampled = sample_with_proportion(df.index.tolist(), confirmed_genes, 0.1)
    
    # Run Pearson on clean and noisy data
    pearson = Pearson(np.transpose(clean_df), '')
    p_values = pearson.values
    np.fill_diagonal(p_values, 0)
    pearson = pd.DataFrame(p_values, index=pearson.columns, columns=pearson.columns)

    # Take subset of pearson data with only the sampled genes
    p_subset = pearson.loc[sampled]
    #print(p_subset)

    # Rank the expression pairs by the absolute value of the pearson correlation
    melted = pd.melt(p_subset.reset_index(), id_vars=['index'], value_vars=p_subset.columns)
    melted.columns = ['input', 'target', 'correlation']

    melted_sorted = melted.sort_values(by='correlation', ascending=False)

    ranked_list = list(melted_sorted.itertuples(index=False, name=None))[:10]
    # Calculate the proportion of confirmed genes in the top k of the ranked pairs
    top_10 = [(input, target) for input, target, _ in ranked_list]
    true_pairs = set(tuple(x) for x in gt.values)
    matches = sum(1 for pair in top_10 if pair in true_pairs)
    proportion = matches / len(top_10)

    print(top_10)
    print(matches)
    print(proportion)

    # Repeat for each dataset

    break


[(6, 14), (57, 43), (9, 24), (91, 62), (41, 1), (57, 1), (57, 14), (20, 14), (9, 14), (6, 11)]
0
0.0
